Aprèn a realitzar web scraping.

Nivell 1
- Exercici 1
Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.



Nivell 2
- Exercici 2
Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.




Nivell 3- Exercici 3
Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 

In [1]:
# imports de llibreries estandard
import pandas as pd
import datetime
import requests
import pprint

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# Adreça pàgina base de la taula
url = 'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [3]:
# Variable perbolcar la pàgina web replicada
page = requests.get(url)

In [4]:
# Generació de l'objecte SOUP retornat pel mètode BS
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Mon, 20 Dec 2021 10:23:06 GMT" data-idioma="esp">
  <meta content="IE=11" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Copyright © BME 2021" id="ctl00_copyright" name="copyright"/>
  <title>
   Bolsa de Madrid - Precios de la sesión
  </title>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" t

In [5]:
# Cerca d e l'objecte amb els atributs concrets "table" 
ibex_table = soup.find("table", attrs={"id": "ctl00_Contenido_tblAcciones"})

In [6]:
# S'emmagatzemen els valors de dins la taula en un objecte
ibex_table_data = ibex_table.find_all("tr")

In [7]:
# Seprant els table headings del cos de dades
headings = []
# Recursivament s'eliminen línies noves i espais blank
for th in ibex_table_data[0].find_all("th"):
    headings.append(th.text.replace('\n', ' ').strip())
print(headings)

# POST: treu llista de capçaleres netes

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [8]:
# Generació de DF amb els índexs de "headings"
dataframe = pd.DataFrame(columns = headings)
dataframe

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora


In [10]:
# Doble bucle. Extracció per files i columnes.
# el bucle de dins recorre columnes i les posa al DF rable data que conté 1 fila
# i el de fora les files fins al final de table_data-1, bolca el DF i  regenera la fila a [] 
table_data = []
for i in range((len(ibex_table_data)-1)): 
    for td in ibex_table_data[i+1].find_all("td"):
        table_data.append(td.text.replace('\n', ' ').strip())
    dataframe.loc[i] = table_data
    table_data = []

In [11]:
# Dataframe final amb les dades d'interès
dataframe

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"156,4000","-1,39","156,7000","154,2000",9.620,"1.495,58",20/12/2021,10:07:34
1,ACERINOX,"10,6450","-2,34","10,6950","10,5250",161.443,"1.717,28",20/12/2021,10:07:36
2,ACS,"20,6300","-5,15","21,1500","20,5800",401.379,"8.379,97",20/12/2021,10:07:57
3,AENA,"128,0000","-3,65","129,6000","127,0500",20.567,"2.648,46",20/12/2021,10:07:11
4,ALMIRALL,"10,8300","-1,81","10,9000","10,7200",52.232,"566,23",20/12/2021,10:05:35
5,AMADEUS,"53,6400","-3,63","54,2600","53,6000",149.400,"8.052,89",20/12/2021,10:08:01
6,ARCELORMIT.,"27,8600","-2,72","28,1700","27,7650",154.828,"4.331,90",20/12/2021,10:07:51
7,B.SANTANDER,"2,6990","-2,74","2,7370","2,6930",7.420.523,"20.117,04",20/12/2021,10:08:01
8,BA.SABADELL,"0,5340","-4,78","0,5468","0,5330",9.527.048,"5.132,84",20/12/2021,10:07:44
9,BANKINTER,"4,1910","-3,05","4,2400","4,1860",303.589,"1.280,52",20/12/2021,10:07:19


In [12]:
# Exportació a csv
dataframe.to_csv('ibex_35.csv')

Exercici 1 amb Selenium ******************************************

<span style='color:blue;font-size:15px'> Es vol accedir a la taula de dades del IBEX 35 desde la pàgina principal. <span>

In [13]:
# Pàgina principal, sobreescric URL
url2 = 'https://www.bolsamadrid.es'

In [14]:
# Es crea una instància per al navegador de Chrome
repositori = 'C:/Users/oriol.gallemi/ITAcademy/chromedriver'
driver = webdriver.Chrome(repositori)

# Accés a web desde webdriver de chrome
driver.get(url2)

In [15]:
# mostra titol de pagina i adreça actual
print(driver.title)
print(driver.current_url)

Bolsa de Madrid
https://www.bolsamadrid.es/esp/aspx/Portada/Portada.aspx


In [16]:
# a "mercados y cotizaciones hi ha l'apartat acciones des d'on s'accedeix a IBEX35. l'objecte és:
acciones = driver.find_element_by_xpath('//div[@class="MenuLink MenuConOp Off"]/a[text()="Acciones"]')
# es fa "click" amb el mètode click() i s'entra a la mateixa pàgina de l'index
acciones.click()
# 'impressio de la pagina actual'
print(driver.current_url)

https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


Es pot accedir asubmenus. PEr exemple als deplegables "Mercado", "Índice" i "sector". Es tria "sector">"petróleo y energía"
Cal fer una sèrie d'accions:

1. seleccionar mercado, índice i sector
2. desplegar cada menú per separat
3. seleccionar opcio a dins de cada desplegable respectivament
4. llançar la consulta directa amb "click" a "consultar"

In [17]:
# Generació de l'objecte "sector" per accedir ala llista desplegable
sector = driver.find_element_by_xpath('//select[@id="SelSector"]/option[text()="Petróleo y Energía"]')
# es faclick() per activar el desplegable
sector.click()

In [18]:
# generació de l'objecte "consultar" per llançar la consulta
consultar = driver.find_element_by_xpath('//input[@id="ctl00_Contenido_Consultar"]')
# "click" per veure els resultats del filtre de la consulta
consultar.click()

In [19]:
# Es procedeix com anteriorment: emmagatzematge de la taula de resultats
table = driver.find_element_by_xpath('//*[@id="ctl00_Contenido_tblAcciones"]/tbody')

In [20]:
# separació de les capçaleres
headings = []

for i in table.find_elements_by_xpath('.//tr'):
    for j in i.find_elements_by_xpath('./th'):
        headings.append(j.get_attribute('textContent'))
print(headings)

['Nombre', 'Mercado', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [21]:
# generació de df per col·locar les dades extretes, amb els headings extrets (llista)
dataframe2 = pd.DataFrame(columns = headings)
dataframe2

,Nombre,Mercado,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora


In [22]:
# Es fa el doble bucle, primer a dins per columnes i després a fora per files
table_data = []
k = 1

for i in table.find_elements_by_xpath('.//tr'):
    for j in i.find_elements_by_xpath('./td'):
        table_data.append(j.get_attribute('textContent').strip())
    if (table_data == []): ## parche files buides
        continue
    if (len(table_data)<dataframe2.shape[1]): ## parche anotacions 
        table_data.append("-")
    dataframe2.loc[k] = table_data # es consolida cada nova fila al df
    k += 1
    table_data = []

In [23]:
# Dataframe final amb les dades d'interès
dataframe2

,Nombre,Mercado,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
1,ACCIONA ENER,M.Continuo,"29,9900","-0,89","30,9000","29,8300",11.785,"353,89",20/12/2021,10:07:08
2,AUDAX RENOV.,M.Continuo,"1,2180","-2,09","1,2460","1,2180",142.017,"174,52",20/12/2021,10:07:51
3,BRASKEM PR.A,Latibex,"8,6500",-,-,-,-,-,17/12/2021,Cierre
4,CEMIG P.,Latibex,"2,1800",-,-,-,-,-,17/12/2021,Cierre
5,COPEL CERT.,Latibex,"4,9000",-,-,-,-,-,17/12/2021,Cierre
6,COPEL ORDINA,Latibex,"0,9350",-,-,-,-,-,17/12/2021,Cierre
7,COPEL PR.B,Latibex,"1,0100",-,-,-,-,-,17/12/2021,Cierre
8,ECOENER,M.Continuo,"4,4000","-0,11","4,5500","4,4000",2.723,"11,99",20/12/2021,09:49:55
9,EDP BRASIL,Latibex,"3,3400",-,-,-,-,-,17/12/2021,Cierre
10,EIDF,MAB - EE,"19,3000",-,-,-,-,-,17/12/2021,Cierre


In [24]:
# tancar sessio navegador
driver.close()

In [25]:
# df2 export a csv amb mètode to_csv
datetoday = datetime.datetime.today().strftime('%Y%m%d')
#os.chdir('/Users/OriolGallemi')
#df.to_csv(open(datetoday+'.csv','w'),index=none) #canvio d'obrir per escriure
dataframe2.to_csv('Petroleo_Energia'+datetoday+'.csv')

# Exercicis documentacio: avaluar en entrega final